In [3]:
import spacy
from spacy import displacy



import spacy_stanza

import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.tag import pos_tag
import re


nlp = spacy.load('en_core_web_sm')
nltk.download('punkt')

!pip install stanza
import stanza

stanza.download('en')
nlpStanza = stanza.Pipeline(lang='en', processors='tokenize,ner,mwt,pos,lemma,depparse')

# Initialize the pipeline
nlpSS = spacy_stanza.load_pipeline("en")

c:\Users\kolvi\AppData\Local\anaconda3\envs\NLP\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kolvi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Der Befehl "pip" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.
2023-08-24 01:10:26 INFO: Downloading default packages for language: en (English) ...
2023-08-24 01:10:27 INFO: File exists: C:\Users\kolvi\stanza_resources\en\default.zip
2023-08-24 01:10:31 INFO: Finished downloading models and saved to C:\Users\kolvi\stanza_resources.
2023-08-24 01:10:31 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES
2023-08

In [4]:
import graphviz
from graphviz import Source

class plotDependencyGraph:
    def __init__(self, name = ""):
        self.subgraph = []
        self.name = name
        self.mainGraph = []
        

    # shall be moved to specific function, here just for convinience and debugging
    # def addSubgraph(self, name, dependency, parserType):
    #     if parserType == "stanza":
    #         head, rel, dependent = dependency
    #         self.addNode(name, head.text, dependent.text, rel)
        
    #     elif parserType == "spacy":
    #         head, dependent, rel = dependency
    #         self.addNode(name, head, dependent, rel)

    def addSubgraph(self, name, head, dependent, rel):
        g = []
        g = graphviz.Digraph(comment= name)
        g.node(head)
        g.node(dependent)
        g.edge(head, dependent, label = rel)
        self.subgraph.append(g)

    def createTree(self):
        self.mainGraph = graphviz.Digraph(comment= self.name)
        for graph in self.subgraph:
            self.mainGraph.subgraph(graph)

        # Render the graph to a PNG file.
        self.mainGraph.render(self.name, view=True, format="png")

In [5]:
def preprocessdict(corpus):
    #get sentences
    sentences = [p for p in corpus.split('\n') if p]    
    return sentences

In [10]:
from collections import defaultdict
import re

functionType = r"<(.*?)>"

class StackStorage:
    def __init__(self, key = "", relation = "", pos = 0):
        self.key = key
        self.relation = relation
        self.pos = pos
        

    def getValues(self):
        return self.key, self.relation, self.pos


    
class RuleBasedExtractor:
    def __init__(self, relations = defaultdict(), words = defaultdict(), rules =[]):
        # current key of the dict
        self.currentWordKey = ""
        # currect relation type
        self.currentRelationType = ""
        # current position of the relation in the current word key dict
        self.currentPositionOfRelIndict = 0
        # maximum position of the List
        self.currentWordKeyMaxRelations = 0


        # depth of key level stored
        # store depth and postion when moving to next position like a stack
        self.depthStack = []
        # depth of key level Integer
        self.depthStackLevel = 0
        
       

        # original relations
        self.words = words
        #original relations
        self.relations = relations

        # rules
        self.rules = rules

        # This indexing is needed when we go deeoer in the stack
        # current exceuting key 
        self.currentExeKey = ""
        # current exceuting relation 
        self.currentExeRelation = ""
        # current exe postions
        self.currentExePostion = 0

        # predicted output string for each significant word key
        self.outputValue = ""
        # predicted relationship type for each string
        self.relationValue = ""


        # flag proceed current execution of certain rule sets due to a failure
        self.proceed = True
        self.checkIfCond = False


        #phrases
        self.phrases = []

    def printPhrases(self):
        for p in self.phrases:
            print(p, sep="\n")


    def execute(self):
        # loop throught the relation keys
        for key in self.relations.keys():
            # store current key
            self.currentWordKey = key
            # get values of the key
            values = self.relations[key]
            # get the max relations assoiated with the key
            self.currentWordKeyMaxRelations = len(values)

            for i, value in enumerate(values):
                # extra layer of error condition
                if i < self.currentWordKeyMaxRelations:
                    self.currentRelationType = value
                    self.currentPositionOfRelIndict = i
                    self.ruleParser()

            self.outputValue += key
            self.phrases.append(self.outputValue)
            self.outputValue = ""



    def getValue(self, key, relation, pos):
        if self.relations[key][pos] == relation:
            return self.words[key][pos]
        
    def returnToRoot(self):
        for i, val in enumerate(self.depthStack):
            self.levelup()
            print("Leveled up")

    
    def functionCalls(self):

        if self.function == "if":
            self.proceed = True
            self.returnToRoot()
            self.checkIfCond = self.when()
            return self.checkIfCond

        elif self.function == "leveldown" and self.checkIfCond:
            self.proceed = self.leveldown()
            return self.proceed

        elif self.function == "levelup" and self.checkIfCond:
           return self.levelup()

        elif self.function == "findIf" and self.checkIfCond:
            self.proceed = self.findIf()
            return self.proceed

        else:
            return False


    def findIf(self):
        #check if relation type is present update the key, level and postion
        # for now only detects first instance
        # multiple instances must be handled
        if self.extractType in self.relations[self.currentExeKey] and self.proceed:
            self.setCurrentExeStates(self.currentExeKey, self.extractType, self.relations[self.currentExeKey].index(self.extractType))
            return True
        else:
            return False
        

    def stripL(self, line):
        return line.strip().strip('\t')

    def ruleParser(self):
        # update current Executing reference
        self.setCurrentExeStates(self.currentWordKey, self.currentRelationType, self.currentPositionOfRelIndict)

        # state machine to extract some infomation based on some rules
        for line in self.rules:
            if self.stripL(line).startswith("<"):
                self.function = re.findall(functionType, line)[0]
                self.extractType = self.stripL(line[line.find('@')+1:].strip().strip('\t'))
                #call the function and set the flag if the extraction needs to be performed or not
                ret = self.functionCalls()

            elif self.stripL(line).startswith("!"):
                relType = self.stripL(line[line.find('!')+1:line.find('@')-1])
                extractType = line[line.find('@')+1:].strip().strip('\t')

                if self.extractType == extractType and self.proceed and ret:
                    if relType == "relation":
                        self.relationValue = self.getValue(self.currentExeKey, self.extractType, self.currentExePostion)    
                    else :
                        self.outputValue += self.getValue(self.currentExeKey, self.extractType, self.currentExePostion) + " "

            else:
                pass

        
    def when(self):
        if self.extractType == self.currentExeRelation:
            return True
        else:
            return False

        
    def levelup(self):
        if self.depthStackLevel > 0:
            # reinstate the old states
            self.degradeToOldState()
            self.depthStack.pop()
            self.depthStackLevel -= 1
            return True
        else:
            return False


    def leveldown(self):
        if self.currentExeKey in self.words.keys():
            # go to new state
            self.depthStack.append(StackStorage(self.currentExeKey, self.currentExeRelation, self.currentExePostion))
            self.depthStackLevel += 1

            # try to update to new state
            flag = self.updateToNewState()

            # in case update failed then revert to lower state back
            if flag == False:
                self.levelup()
                return False
            
            return True
                

    def setCurrentExeStates(self, key = "", relation = "", pos = 0):
        self.currentExeKey = key
        self.currentExeRelation =  relation
        self.currentExePostion = pos


    def updateToNewState(self):
        newKey = self.words[self.currentExeKey][self.currentExePostion]
        if newKey in self.words.keys():
            # relation type and postion will be checked later on depending on condition
            self.setCurrentExeStates(newKey)
            return True
        else:
            return False

    def degradeToOldState(self):
        key, relation, pos = self.depthStack[-1].getValues()
        self.setCurrentExeStates(key, relation, pos)



In [16]:
from IPython.core.display import display_html, HTML
from pathlib import Path

from collections import defaultdict

def evaluteRules(words, relations, rules):
    rulechunks = []
    wordchunks = []
    for line in rules:
        if line.startswith("--start"):
            rule = []
        elif line.startswith("@"):
            rule.append(line[1:].strip())
        elif line.startswith("--end"):
            rulechunks.append(rule)
        else:
            pass
    wordchunks = getWord(rulechunks, words, relations)
    return wordchunks

def getWord(rulechunks, words, relations):
    wordchunks = []
    stringVal = ""
    for key in relations.keys():
        chunks = []
        stringVal += key
        for rule in rulechunks:
            if len(rule) > 1:
                pos = [relations[key].index(subrule)  for subrule in rule if subrule in relations[key]]
                for idx in pos:
                    if words[key][idx] not in stringVal:
                        stringVal = words[key][idx] + " " + stringVal
                if len(pos) > 0:
                    chunks.append(stringVal)

            else:
                if rule[0] in relations[key] :
                    idxs = [i for i, j in enumerate(relations[key]) if j == rule[0]]
                    [chunks.append(words[key][idx] + " " + stringVal) for idx in idxs if words[key][idx] not in stringVal]
                        

        if chunks != []:
            wordchunks.append(chunks)
        stringVal = "" 
                
    return wordchunks
    

def drawRelationshipsStanza(sentence,  plot = plotDependencyGraph("Trial"), rules = []):
    #stanford parser
    docStanza = nlpStanza(sentence)
    words = defaultdict(list)
    relations = defaultdict(list)
   
    data = docStanza.to_dict()

    # loop through sentences and its dependencies and add subgraph

    # print(*[f'id: {word.id}\tword: {word.text}\thead id: {word.head}\thead: {sent.words[word.head-1].text if word.head > 0 else "root"}\tdeprel: {word.deprel}' for sent in docStanza.sentences for word in sent.words], sep='\n')
    for sent in docStanza.sentences:
        for dependency in sent.dependencies:
            head, rel, dependent = dependency

            words[head.text].append(dependent.text)
            relations[head.text].append(rel)
            #print(head.id)
            name = head.text + "_stanza_" + dependent.text
           
            # print(dependent.text, "<---" , head.text," = " ,rel, (head.text, dependent.text))
            plot.addSubgraph(name, head.text, dependent.text, rel)

    # for dataList in evaluteRules(words, relations, rules):
    #     for val in dataList:
    #         print(val)

    rbe = RuleBasedExtractor(relations, words, rules)
    rbe.execute()
    rbe.printPhrases()

    pass
    # combing spacy and stanza
    doc = nlpSS(sentence)
    html = displacy.render(doc, style="dep", jupyter=False)
    # display_html(HTML(html))
    output_path = Path("./images/dependency_plot.svg") # you can keep there only "dependency_plot.svg" if you want to save it in the same folder where you run the script 
    output_path.open("w", encoding="utf-8").write(html)

In [17]:
import os
# os.environ["PATH"] += os.pathsep + r"C:\Users\kolvi\AppData\Local\anaconda3\envs\NLP\Library\bin"
# print(os.environ["PATH"])
import pydot



data = open("dict_en_rule2.txt", "r").read()
# data1 = open("try.txt", 'rb+').read()
rules = preprocessdict(data)

testStr = "The condition that CSWS transitions from system OFF state to system ON state are as follows"

plot = plotDependencyGraph("plot")
drawRelationshipsStanza(testStr, plot, rules)
plot.createTree()

condition
follows
CSWS transitions
system OFF state
system
ROOT


In [ ]:

#DependencyGraph.tree()

def getRelationshipsStanza(sentence):
    docStanza = nlpSS(sentence)

    for name in docStanza.noun_chunks:
        print(name)
        print("\n")

    pass


getRelationshipsStanza("The condition that CSWS transitions from system OFF state to system ON state are as follows")
    

CSWS transitions




### import teminology
a coordinating conjunction, clause, passive, NP, ajective, adverb, 

### acomp
An adjectival complement of a verb is an adjectival phrase which functions as the complement (like an
object of the verb).

Sue asked George to respond to her offer, respond <--- asked  =  xcomp ('asked', 'respond')\

### advcl
An adverbial clause modifier of a VP or S is a clause modifying the verb (temporal clause, consequence,
conditional clause, purpose clause, etc.).

"The accident happened as the night was falling" , falling <--- happened  =  advcl ('happened', 'falling')\
If you know who did it, you should tell the teacher, know <--- tell  =  advcl ('tell', 'know')\
He talked to him in order to secure the account, secure <--- talked  =  advcl ('talked', 'secure')\

### advmod
An adverb modifier of a word is a (non-clausal) adverb or adverb-headed phrase that serves to modify
the meaning of the word.

“Genetically modified food, Genetically <--- modified  =  advmod ('modified', 'Genetically')\
less often", less <--- often  =  advmod ('often', 'less')\

### agent
An agent is the complement of a passive verb which is introduced by the preposition “by” and does the
action. This relation only appears in the collapsed dependencies, where it can replace prep by, where
appropriate. It does not appear in basic dependencies output.

The man has been killed by the police, police <--- killed  =  obl:agent ('killed', 'police')\
Effects caused by the protein are important, protein <--- caused  =  obl:agent ('caused', 'protein')\

### amod
An adjectival modifier of an NP is any adjectival phrase that serves to modify the meaning of the NP.

Sam eats red meat, red <--- meat  =  amod ('meat', 'red')\

### appos : appositional modififer
An appositional modifier of an NP is an NP immediately to the right of the first NP that serves to define
or modify that NP. It includes parenthesized examples, as well as defining abbreviations in one of these
structures.

Sam, my brother, arrived, brother <--- Sam  =  appos ('Sam', 'brother')\
THe australian broascasting cooperation \"ABC\", ABC <--- cooperation  =  appos ('cooperation', 'ABC')\

### aux  auxillary
An auxiliary of a clause is a non-main verb of the clause, e.g., a modal auxiliary, or a form of “be”, “do”
or “have” in a periphrastic tense.

Reagan has died, has <--- died  =  aux ('died', 'has') \
He should leave, should <--- leave  =  aux ('leave', 'should') \
The system must transit from state A to state B, must <--- transit  =  aux ('transit', 'must')\

### auxpass: passive auxiliary
A passive auxiliary of a clause is a non-main verb of the clause which contains the passive information.

Kennedy has been killed,  has <--- killed  =  aux ('killed', 'has'), been <--- killed  =  aux:pass ('killed', 'been') \
"Kennedy was/got killed", was <--- killed  =  aux:pass ('killed', 'was'), aux:pass ('killed', 'got')

### cc: coordination
A coordination is the relation between an element of a conjunct and the coordinating conjunction word of
the conjunct. (Note: different dependency grammars have different treatments of coordination. We take
one conjunct of a conjunction (normally second) as the head of the conjunction.) A conjunction may
also appear at the beginning of a sentence. This is also called a cc, and dependent on the root predicate
of the sentence.

They either ski or snowboard \
either <--- ski  =  cc:preconj ('ski', 'either')\
or <--- snowboard  =  cc ('snowboard', 'or')\

###  conj: conjunct
A conjunct is the relation between two elements connected by a coordinating conjunction, such as “and”,
“or”, etc. We treat conjunctions asymmetrically: The head of the relation is the first conjunct and other
conjunctions depend on it via the conj relation.

They either ski or snowboard, snowboard <--- ski  =  conj ('ski', 'snowboard')

###  cop: copula
A copula is the relation between the complement of a copular verb and the copular verb.